# Modelleringsdel för kunskapskontroll 2


## Steg 1. Importera nödvändiga paket.

In [1]:
# Paket för datahantering
import numpy as np
import pandas as pd

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Preprocessing/pipelin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Modeller 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier 
    
# För export av modellen/scalern för vidare användning i Streamlit-appen
from joblib import dump

## Steg 2. Läs in MNISt-datasetet och splitta det. 
*//Jag är en visare man nu och splittar direkt//*

In [8]:
# Läs in hela MNIST
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splittar 80/20 för träning (och korsvalidering) respektive test
# Stratifiering garanterar statistisk säkerhet med jämn fördelning av siffrorna i båda seten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

"""# För att snabba upp hanteringen och slippa köra StandardScaler normaliserar vi data
# Genom att dela med 255 hamnar allt mellan 0 och 1 som float64.
X_train = X_train / 255.0
X_test = X_test / 255.0
"""

print(f"Träningsset: {X_train.shape}")
print(f"Testset: {X_test.shape}")

Träningsset: (56000, 784)
Testset: (14000, 784)


## Steg 3. Dags att börja testa olika modeller. Jag har till att börja med valt en linjär modell, en icke-linjär modell och en trädmodell. 

Vi börjar med att skapa en dict för resultaten

In [9]:
results = {}

Linjär modell: SGDClassifier

In [5]:
# Pipeline
sgd_pipe = Pipeline([
    ('sgd', SGDClassifier(random_state=42))
])

# Param_grid: log_loss ger en logistisk regression som "baseline". 
# Vi ser vilken Alpha som ger bäst regularisering 
param_grid_sgd = {
    'sgd__loss': ['log_loss'],
    'sgd__alpha': [0.0001, 0.01],
    'sgd__max_iter': [1000]
}

# GridSearchCV: CV=3 ger bra balans mellan tid och statistisk säkerhet
sgd_grid = GridSearchCV(
    estimator=sgd_pipe,
    param_grid=param_grid_sgd,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Träning av modellen 
sgd_grid.fit(X_train, y_train)

# Resultat
print(f"Bästa parametrar: {sgd_grid.best_params_}")
print(f"Bästa genomsnittliga CV-accuracy: {sgd_grid.best_score_:.4f}")

# Spara resultaten
results["SGD (Baseline)"] = {
    "Best Accuracy": round(sgd_grid.best_score_, 4),
    "Best Params": sgd_grid.best_params_
}


Fitting 3 folds for each of 2 candidates, totalling 6 fits
Bästa parametrar: {'sgd__alpha': 0.0001, 'sgd__loss': 'log_loss', 'sgd__max_iter': 1000}
Bästa genomsnittliga CV-accuracy: 0.9110


En icke-linjär modell. SVC har jag hört gott om. 

In [ ]:
# Den här modellen tog lång tid. Jag försöker uppskatta dess kvalitet med ett mindre urval
X_train_svc_sub, _, y_train_svc_sub, _ = train_test_split(
    X_train, y_train, train_size=10000, stratify=y_train, random_state=42
)

# Pipeline
svc_pipe = Pipeline([
    ('svc', SVC(kernel='rbf', random_state=42))
])

# Param_grid
param_grid_svc = {
    'svc__C': [1, 10],
    'svc__gamma': ['scale', 0.01]
}

# GridSearchCV
svc_grid = GridSearchCV(
    estimator=svc_pipe,
    param_grid=param_grid_svc,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2  # Mer info så det blir mindre trist att vänta
)

# Träning på det mindre urvalet
svc_grid.fit(X_train_svc_sub, y_train_svc_sub)

# 6. Resultat och sparande
print(f"Bästa SVC-parametrar: {svc_grid.best_params_}")
print(f"Bästa SVC CV-accuracy: {svc_grid.best_score_:.4f}")

results["SVC (Non-linear)"] = {
    "Best Accuracy": round(svc_grid.best_score_, 4),
    "Best Params": svc_grid.best_params_
}

Fitting 3 folds for each of 4 candidates, totalling 12 fits


En trädmodell - Extra Trees fick bra resultat i boken så den kör vi med.

In [ ]:
# Pipeline
extra_trees_clf = ExtraTreesClassifier(random_state=42)

pipeline = Pipeline([
    ("model", extra_trees_clf)
])

# Param_grid
param_grid = {
    "model__n_estimators": [200, 250, 300]
}

# GridSearchCV
extra_trees_grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,  
    verbose=1   
)

# Träning
extra_trees_grid.fit(X_train, y_train)

# Spara resultat
extra_trees_accuracy = extra_trees_grid.best_score_
extra_trees_params = extra_trees_grid.best_params_

results["Extra Trees (med Scaler)"] = {
    "Best Accuracy": round(extra_trees_accuracy, 4),
    "Best Params": extra_trees_params
}

print(f"Bästa parametrar: {extra_trees_params}")
print(f"Bästa CV-accuracy: {extra_trees_accuracy:.4f}")

Tränar Extra Trees med StandardScaler (Kompisens version)...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Bästa parametrar: {'model__n_estimators': 300}
Bästa CV-accuracy: 0.9700
